# ChatPromptTemplate으로 역할별 메시지 구성하기

이 노트북에서는 **ChatPromptTemplate**을 사용하여 역할별 메시지를 구조화하는 방법을 알아봅니다.

## PromptTemplate vs ChatPromptTemplate

| 구분 | PromptTemplate | ChatPromptTemplate |
|------|----------------|-------------------|
| 구조 | 단일 문자열 | 메시지 리스트 |
| 역할 구분 | 없음 | system, human, ai 역할 지원 |
| 사용 모델 | LLM (텍스트 완성) | Chat Model (대화형) |
| 출력 | StringPromptValue | ChatPromptValue |

## ChatPromptTemplate의 장점

1. **역할 분리**: System 지시사항과 User 입력을 명확히 구분
2. **대화 흐름 구성**: 여러 턴의 대화를 자연스럽게 표현
3. **Chat Model 최적화**: ChatOllama, ChatOpenAI 등과 완벽 호환
4. **가독성**: 복잡한 프롬프트를 구조적으로 관리

## 메시지 타입 (튜플 형식)

```python
('system', '시스템 지시사항')   # AI 역할/규칙 설정
('human', '사용자 입력')        # 사용자 메시지
('ai', 'AI 응답')              # AI 이전 응답 (few-shot 등)
```

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (2,278 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current u

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. ChatPromptTemplate 코드 실행

**코드 구조 설명:**

### 1) ChatPromptTemplate 생성
```python
template = ChatPromptTemplate.from_messages([
    ('system', '시스템 지시사항...'),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
```
- `from_messages()` - 튜플 리스트로 메시지 정의
- 첫 번째 요소: 역할 (`'system'`, `'human'`, `'ai'`)
- 두 번째 요소: 메시지 내용 (변수 `{context}`, `{question}` 포함 가능)

### 2) 메시지 구조 분석
```
┌─────────────────────────────────────────────────────┐
│ system: "Context를 기반으로 질문에 대답하세요..."      │  ← AI 행동 규칙
├─────────────────────────────────────────────────────┤
│ human: "Context: {context}"                         │  ← 참고 자료
├─────────────────────────────────────────────────────┤
│ human: "Question: {question}"                       │  ← 실제 질문
└─────────────────────────────────────────────────────┘
```

### 3) LCEL 체인 & 실행
```python
chain = template | model
response = chain.invoke({'context': '...', 'question': '...'})
```
- 변수 값을 딕셔너리로 전달
- `response.content`로 텍스트 응답 추출

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

template = ChatPromptTemplate.from_messages([
    ('system', '''아래 작성한 컨텍스트(Context)를 기반으로
    질문(Question)에 대답하세요. 제공된 정보로 대답할 수 없는 질문이라면 "모르겠어요." 라고 답하세요.'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])

# LLM 모델
model = ChatOllama(model='llama3.2')

# 체인 연결
chain = template | model

# 실행
response = chain.invoke({
    'context': '''거대 언어 모델(LLM)은 자연어 처리(NLP) 분야의 최신 발전을 이끌고 있습니다.
    거대 언어 모델은 더 작은 모델보다 우수한 성능을 보이며, NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게
    매우 중요한 도구가 되었습니다. 개발자들은 Hugging Face의 `transformers` 라이브러리를
    활용하거나, `openai` 및 `cohere` 라이브러리를 통해 OpenAI와 Cohere의 서비스를 이용하여
    거대 언어 모델을 활용할 수 있습니다.
    ''',
    'question': '거대 언어 모델은 어디서 제공하나요?'
})

print(response.content)

거대 언어 모델은 Hugging Face의 `transformers` 라이브러리나, OpenAI 및 Cohere의 서비스를 통해 제공하거나, 직접 개발하여 사용할 수 있습니다.


---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai import ChatOpenAI
model = ChatOpenAI()

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## PromptTemplate vs ChatPromptTemplate 코드 비교

### PromptTemplate (04.prompt.ipynb)
```python
template = PromptTemplate.from_template('''
Context: {context}
Question: {question}
Answer: ''')
```
- 단일 문자열, 역할 구분 없음

### ChatPromptTemplate (이 노트북)
```python
template = ChatPromptTemplate.from_messages([
    ('system', '지시사항...'),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
```
- 역할별 메시지 분리, Chat Model에 최적화

## 다양한 ChatPromptTemplate 패턴

### Few-shot 예시 포함
```python
ChatPromptTemplate.from_messages([
    ('system', '영어를 한국어로 번역하세요.'),
    ('human', 'Hello'),
    ('ai', '안녕하세요'),  # 예시 응답
    ('human', '{input}'),  # 실제 입력
])
```

### 멀티턴 대화
```python
ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 상담원입니다.'),
    ('human', '{previous_question}'),
    ('ai', '{previous_answer}'),
    ('human', '{current_question}'),
])
```